### LLM OpenAeroStruct

#### A multiagent tool to write optimization code for OpenAeroStruct with simply high level inputs from the user

In [1]:
# Import the generative ai library
import google.genai as genai
import ollama

# Import all the modules necessary to run OpenAeroStruct and paths
import re
import time
import os
import subprocess
import warnings
import numpy as np
import pandas as pd 
import openmdao.api as om
import json
import shutil
import platform
from datetime import datetime

# import OpenAeroStruct modules
from openaerostruct.meshing.mesh_generator import generate_mesh
from openaerostruct.geometry.geometry_group import Geometry
from openaerostruct.aerodynamics.aero_groups import AeroPoint

# Import the plotting libraries
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import niceplots  # Optional but recommended

# Import the LaTeX libraries used for report format conversions
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from io import StringIO

#Ignore warnings and use the nice plots style
warnings.filterwarnings("ignore")

plt.style.use(
    niceplots.get_style("james-dark")
)  # Options: "doumont-light", "doumont-dark", "james-light", "james-dark"

In [2]:
import sys
sys.path.append("../openaerostruct-llm")

from llm_functions.agents import (
    ReformulatorAgent, 
    BaseMeshAgent, 
    GeometryAgent, 
    OptimizerAgent, 
    ResultsReaderAgent, 
    ReportWriter
)

In [3]:
# Define the initial sample query from the user.
User_Request = """For this design, we will keep the area at S = 100 m2. The span is b = 10 m. The cruise condition corresponds to CL = 2.0. Your job is to minimize drag at the condition of CL = 2.0 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

#HERE ARE SOME BACKUP PROMPTS
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area at S = 100 m2. The span is b = 10 m. The cruise condition corresponds to CL = 2.0. Your job is to minimize drag at the condition of CL = 2.0 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, dihedral, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

In [4]:
error_gen_flag = False
while not error_gen_flag:
    reformulator = ReformulatorAgent()
    reformulator_output = reformulator.execute_task(User_Request)

    # SAFE CHECKS
    if (reformulator_output is not None and 
        isinstance(reformulator_output, dict) and
        all(key in reformulator_output for key in ["objective_function", "design_variables", "trim_condition"])):
        error_gen_flag = True
        print("Reformulator output generated successfully")
    else:
        print("Invalid reformulator output, retrying...")
        print(f"Output: {reformulator_output}")


Reformulator output generated successfully


In [5]:
print("Reformulator Output:")
print(reformulator_output)

Reformulator Output:
{'objective_function': 'Minimize drag', 'trim_condition': 'CL = 2.0', 'geometric_constraint': 'Wing area (S) = 100 m2, Span (b) = 10 m', 'design_variables': 'Taper, Twist, Sweep', 'baseline_wing_mesh': 'Rectangular mesh', 'optimization_algorithm': 'SLSQP', 'plotting_requirements': 'Plot of the elliptical lift distribution', 'errors': 'None'}


#### Now start writing the file into RunOAS to prepare for the optimization and plotting script using LLM.

In [6]:
# Write the mesh prompt
error_gen_flag = False
while not error_gen_flag:
    MeshPrompt = f"""For this wing design, the geometric parameters are as follows: {reformulator_output["geometric_constraint"]}, and the type of the wing mesh should be: {reformulator_output["baseline_wing_mesh"]}"""
    mesher = BaseMeshAgent()
    mesher_output = mesher.execute_task(MeshPrompt)

    # SAFE CHECKS
    if (mesher_output is not None and 
        isinstance(mesher_output, dict) and 
        "python_code" in mesher_output and 
        mesher_output["python_code"] is not None):
        error_gen_flag = True
        print("Mesh code generated successfully")
    else:
        print("Invalid mesh output, retrying...")
        print(f"Output: {mesher_output}")


Mesh code generated successfully


In [7]:
print(mesher_output["python_code"])
print(mesher_output)

mesh_dict = {
    "num_y": 19, #number of panels in the y direction, 19 is a good starting number
    "num_x": 3, #number of panels in the x direction, 3 is a number
    "wing_type": "rect", #This can either be "rect" or "crm" only
    "symmetry": True, # True if the wing is symmetric, False if it is not, wings are typically symmetric
    "span": 10.0, #This is the full span of the wing in meters
    "root_chord": 10.0, #This is the root chord of the wing in meters
    "span_cos_spacing": 0.0, #This is usually not edited
    "chord_cos_spacing": 0.0, #This is usually not edited
}

# Generate VLM mesh for half-wing
mesh = generate_mesh(mesh_dict)   # this creates a rectangular wing mesh, DO NOT EDIT THIS LINE

# plot mesh
plot_mesh(mesh)  #this plots the rectangular wing mesh, DO NOT EDIT THIS LINE
{'calculations_and_explain': "The task requires generating a mesh for a rectangular wing with a given area and span. The root chord needs to be calculated using the formula: root_chord = area

In [8]:
# Geometry Setup
error_gen_flag = False
while not error_gen_flag:
    GeometryPrompt = f"""For this wing design, we are allowed to change the following parameters: {reformulator_output["design_variables"]}"""
    geometry_setup = GeometryAgent()
    geometry_output = geometry_setup.execute_task(GeometryPrompt)

    # SAFE CHECKS
    if (geometry_output is not None and 
        isinstance(geometry_output, dict) and 
        "python_code" in geometry_output and 
        geometry_output["python_code"] is not None):
        error_gen_flag = True
        print("Geometry code generated successfully")
    else:
        print("Invalid geometry output, retrying...")
        print(f"Output: {geometry_output}")

Geometry code generated successfully


In [9]:
print(geometry_output["python_code"])
print(geometry_output)

surface = {
    # Wing definition, KEEP THE SAME UNLESS ASKED TO CHANGE
    "name": "wing",  # name of the surface, keep as wing
    "symmetry": True,  # if true, model one half of wing reflected across the plane y = 0
    "S_ref_type": "wetted",  # how we compute the wing area, can be 'wetted' or 'projected'
    "mesh": mesh,

    # Aerodynamic performance of the lifting surface at an angle of attack of 0 (alpha=0).
    # These CL0 and CD0 values are added to the CL and CD obtained from aerodynamic analysis of the surface to get the total CL and CD.
    # These CL0 and CD0 values do not vary wrt alpha. DO NOT EDIT THEM UNLESS ASKED TO.
    "CL0": 0.0,  # CL of the surface at alpha=0
    "CD0": 0.0,  # CD of the surface at alpha=0

    # Airfoil properties for viscous drag calculation, DO NOT CHANGE UNLESS ASKED TO
    "k_lam": 0.05,  # percentage of chord with laminar flow, used for viscous drag
    "c_max_t": 0.303,  # chordwise location of maximum (NACA0015)
    "t_over_c_cp": np.ar

In [10]:
# Optimization Setup
error_gen_flag = False
while not error_gen_flag:
    OptimizerPrompt = f"""For this wing design, the optimization parameters are as follows: {reformulator_output["design_variables"]}, the objective function is {reformulator_output["objective_function"]}, the geometric constraints are {reformulator_output["geometric_constraint"]}, the flight condition is {reformulator_output["trim_condition"]}, and the optimization algorithm is {reformulator_output["optimization_algorithm"]}"""
    optimizer_setup = OptimizerAgent()
    optimizer_output = optimizer_setup.execute_task(OptimizerPrompt)

    # SAFE CHECKS
    if (optimizer_output is not None and 
        isinstance(optimizer_output, dict) and 
        "python_code" in optimizer_output and 
        optimizer_output["python_code"] is not None):
        error_gen_flag = True
        print("Optimizer code generated successfully")
    else:
        print("Invalid optimizer output, retrying...")
        print(f"Output: {optimizer_output}")


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 20.191633804s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 

In [ ]:
print(optimizer_output["python_code"])
print(optimizer_output)

In [ ]:
import textwrap

# Write the chunks of code into the template file and run it
template_file = "RunOAS_template.py"

def dedent_code(code_str):
    """Fully dedent code using Python standard library."""
    return textwrap.dedent(code_str).strip()

with open(template_file, "r") as file:
    template_code = file.read()
    
    # Replace all placeholders with dedented generated code
    template_code = template_code.replace(
        '"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""',
        f'"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""\n{textwrap.dedent(mesher_output["python_code"]).strip()}'
    )
    template_code = template_code.replace(
        '"""Part 2:  DO THE GEOMETRY SETUP HERE"""',
        f'"""Part 2:  DO THE GEOMETRY SETUP HERE"""\n{textwrap.dedent(geometry_output["python_code"]).strip()}'
    )
    template_code = template_code.replace(
        '"""Part 3: PUT THE OPTIMIZER HERE """',
        f'"""Part 3: PUT THE OPTIMIZER HERE """\n{textwrap.dedent(optimizer_output["python_code"]).strip()}'
    )

# Write the modified code to a new file
run_oas_file = "RunOAS.py"
with open(run_oas_file, "w") as file:
    file.write(template_code)

print("Created RunOAS.py file with all generated code sections")


In [ ]:
# Now run the script using a subprocess, also capture all the outputs, and save it as a text file.
output_file = "openaerostruct_out/output.txt"
with open(output_file, "w") as file:
    # Run the script and capture the output
    process = subprocess.Popen(["python3", run_oas_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decode the output and write it to the file
    file.write(stdout.decode())
    file.write(stderr.decode())

### After Running the Optimization, the LLM should then have access to the plots then analyze the results

In [ ]:
HTML_Report = "./openaerostruct_out/RunOAS_out/reports/opt_report.html"
pdf_output_path = "./figures/Opt_History.pdf"

if not os.path.exists(HTML_Report):
    print(f"HTML missing: {HTML_Report}")
else:
    os.makedirs(os.path.dirname(pdf_output_path), exist_ok=True)
    
    texbin = "/Library/TeX/texbin" if os.uname().sysname == "Darwin" else ""
    env = os.environ.copy()
    if texbin:
        env['PATH'] = f"{texbin}:{env.get('PATH', '')}"
    
    result = subprocess.run([
        "pandoc", HTML_Report, "-o", pdf_output_path,
        "--pdf-engine=xelatex",
        "-V", "geometry:landscape,a4paper",
        "-V", "geometry:margin=5mm",
        "-V", "geometry:includeheadfoot",
        "--variable", "fontsize=5pt"
        # No mainfont=Arial
    ], env=env, capture_output=True, text=True, timeout=180, shell=False)
    
    if result.returncode == 0:
        print(f"PDF: {pdf_output_path}")
    else:
        print(f"Return code: {result.returncode}")
        print(f"Stdout: {result.stdout}")
        print(f"Stderr: {result.stderr}")


In [ ]:
def run_plot_wing(file_path):
    """
    Execute the plot_wing command on a specified aero.db file
    
    Args:
        file_path (str): Path to the aero.db file
    """
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        return
        
    try:
        # Run plot_wing command and capture output
        result = subprocess.run(["plot_wing", file_path], 
                               capture_output=True, 
                               text=True, 
                               check=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")
    except FileNotFoundError:
        print("Error: 'plot_wing' command not found. Please ensure it's installed and in your PATH and follow the changes to the plot_wing function shown in readme.")

# Example usage
file_path = "./openaerostruct_out/RunOAS_out/aero.db"
run_plot_wing(file_path)

In [ ]:
# Results Reading
error_gen_flag = False
while not error_gen_flag:
    ResultsPrompt = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, and the optimization results are as follows: {optimizer_output}"""
    results_setup = ResultsReaderAgent()
    results_output = results_setup.execute_task(ResultsPrompt)

    # SAFE CHECKS
    if (results_output is not None and 
        results_output["Analysis"] is not None):
        error_gen_flag = True
        print("Results analysis generated successfully")
    else:
        print("Invalid results output, retrying...")
        print(f"Output: {results_output}")


In [ ]:
results_output

In [ ]:
# Report Writing
error_gen_flag = False
while not error_gen_flag:
    ReportPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, the analysis by the LLM is {results_output}"""
    report_setup = ReportWriter()
    report_output = report_setup.execute_task(ReportPrompt)

    # SAFE CHECKS
    if (report_output is not None and 
        report_output["ReportText"] is not None):
        error_gen_flag = True
        print("Report generated successfully")
    else:
        print("Invalid report output, retrying...")
        print(f"Output: {report_output}")

In [ ]:
# Generate filename with current date format YYMMDDHH (e.g., 26010121_Report.tex)
now = datetime.now()
date_str = now.strftime("%y%m%d%H")
report_file = f"././report_outputs/{date_str}_Report.tex"

# Create outputs directory if it doesn't exist
os.makedirs("././report_outputs", exist_ok=True)

with open(report_file, "w", encoding="utf-8") as file:
    file.write(report_output["ReportText"])

print(f"Report saved as: {report_file}")